In [ ]:
import numpy as np
from scipy.interpolate import RectBivariateSpline
import xarray as xr

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

import pandas as pd
from matplotlib import pyplot

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

# Visualise data

In [ ]:
fp = r'data/elevation/ETOPO1_Bed_g_geotiff.tif'
img = rasterio.open(fp)

fig, ax = pyplot.subplots(1,1, figsize=(12,7))
show(img, ax=ax, cmap='gist_earth', title='Precipitation')
pyplot.show()

# Clip around Sundaland 

In [ ]:
out_tif = r'data/elevation/elev_clip.tif'
minx, miny = 80., -20.00
maxx, maxy = 140., 20.

In [ ]:
bbox = box(minx, miny, maxx, maxy)
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
geo = geo.to_crs(crs={'init': 'epsg:4326'})
coords = getFeatures(geo)
out_img, out_transform = mask(img, shapes=coords, crop=True)
out_meta = img.meta.copy()
epsg_code = int('4326')
out_meta.update({"driver": "GTiff",
                "height": out_img.shape[1],          
                 "width": out_img.shape[2],
                 "transform": out_transform,
                 "crs": pycrs.parse.from_epsg_code(epsg_code).to_proj4()}                 
               )

with rasterio.open(out_tif, "w", **out_meta) as dest:
    dest.write(out_img)

In [ ]:
tmp = rasterio.open('data/elevation/elev_clip.tif')
fig, ax = pyplot.subplots(1,1, figsize=(12,7))
show(tmp, ax=ax, cmap='gist_earth', title='Elevation')
pyplot.show()

## Convert to UTM

UTM zone 49N - EPSG:32649

In [ ]:
dst_crs = 'EPSG:32649'
out_utm = r'data/elevation/clip_utm.tif'

In [ ]:
with rasterio.open(out_tif) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(out_utm, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)
            
clipped_utm = rasterio.open(out_utm)
fig, ax = pyplot.subplots(1,1, figsize=(12,7))
show(clipped_utm, ax=ax, cmap='gist_earth', title='Elevation')
pyplot.show()

## Clip based on UTM coordinates

In [ ]:
minux, minuy = -1800000., -1400000.
maxux, maxuy = 3000000., 2200000.

out_tif_utm2 = r'data/elevation/clip_utm1.tif'

In [ ]:
bbox2 = box(minux, minuy, maxux, maxuy)
geo2 = gpd.GeoDataFrame({'geometry': bbox2}, index=[0], crs=from_epsg(32649))
geo2 = geo2.to_crs(crs=clipped_utm.crs.data)

coords2 = getFeatures(geo2)
#print(coords2)

out_utm2, out_transform2 = mask(clipped_utm, shapes=coords2, crop=True)
out_meta2 = clipped_utm.meta.copy()
epsg_code2 = int(clipped_utm.crs.data['init'][5:])
# print(out_meta2)
# print(epsg_code2)

out_meta2.update({"driver": "GTiff",
                "height": out_utm2.shape[1],          
                 "width": out_utm2.shape[2],
                 "transform": out_transform2,
                 "crs": pycrs.parse.from_epsg_code(epsg_code2).to_proj4()}                 
               )

with rasterio.open(out_tif_utm2, "w", **out_meta2) as dest:
    dest.write(out_utm2)
    
clipped = rasterio.open(out_tif_utm2)
fig, ax = pyplot.subplots(1,1, figsize=(12,7))
show(clipped, ax=ax, cmap='gist_earth', title='Elevation')
pyplot.show()

# Create a regular grid for badlands

In [ ]:
# Read the data
da = xr.open_rasterio(out_tif_utm2)
x = da['x']
y = np.flipud(da['y'])
dx = x[1]-x[0]
dy = y[1]-y[0]

elev = np.flipud(da.values[0,1:-1,1:-1])

X, Y = np.meshgrid(x[1:-1], y[1:-1])

interp_spline = RectBivariateSpline(y[1:-1], x[1:-1], elev)


print('dx,dy',dx,dy)
print('xmin,xmax',X.min(),X.max())
print('ymin,ymax',Y.min(),Y.max())

In [ ]:
dx2 = 5000.
dy2 = 5000.

rxmin = -1790000.
rxmax = 2990000.
rymin = -1395000.
rymax = 2195000.

In [ ]:
x2 = np.arange(rxmin, rxmax, dx2)
y2 = np.arange(rymin, rymax, dy2)
X2, Y2 = np.meshgrid(x2, y2)
Z2 = interp_spline(y2, x2)

In [ ]:
df = pd.DataFrame(
    {
        "X": X2.flatten(),
        "Y": Y2.flatten(),
        "elev": Z2.flatten(),
    }
)

df.to_csv(
    "data/elev5km.csv",
    columns=["X", "Y", "elev"],
    sep=" ",
    index=False,
    header=0,
)